In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
url = 'https://www.espn.com/golf/leaderboard'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
response = requests.get(url, headers=headers)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the table with the specified class
    table = soup.find("tbody", class_="Table__TBODY")
    
    # Check if the table was found
    
    # Extract the rows of the table
    rows = table.find_all("tr")
    
    # Initialize an empty list to store the table data
    data = []
    
    # Loop through each row and extract the data
    for row in rows:
        # Extract the cells (td) of the row
        cells = row.find_all("td")
        
        # Extract the text content of each cell and append to the data list
        row_data = [cell.get_text() for cell in cells]
        data.append(row_data)
    headers = soup.find_all('th')
    header_texts = [header.text.strip() for header in headers]
    # Convert the data list into a pandas DataFrame
    df = pd.DataFrame(data)
    available_columns = [col for col in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] if col in df.columns]
    df = df[available_columns]
    df.columns = [x for x in header_texts]
    
    # df['R3'] = df['R3'].replace('--', '0')
    # df['R4'] = df['R4'].replace('--', '0')

    # Filter DataFrame
    df = df.dropna(subset = ['PLAYER'])
    filtered_df = df[~df['PLAYER'].str.endswith('(a)')]
    filtered_df = filtered_df[filtered_df['SCORE'] != 'CUT']
    filtered_df["TODAY"] = filtered_df["TODAY"].replace({
            'E': '0', 
        })
    filtered_df["R3"] = filtered_df["R3"].replace({
            'E': '0', 
            '--': '0'
        })

    for idx, row in filtered_df.iterrows(): 
            filtered_df.at[idx, 'TODAY'] = int(str(row['TODAY']).strip('+'))
            filtered_df.at[idx, 'R3'] = int(str(row['R3']).strip('+'))
            
    max_today = filtered_df['TODAY'].max() 
    max_r3 = filtered_df['R3'].max()

    # If we are on Saturday (R3) only add R1 and R2 else add R1 R2 and R3 
    current_date = datetime.now().date().strftime('%d-%m-%Y')

    for idx, row in df.iterrows(): 
        if row['SCORE'] == 'CUT': 
            if current_date == '12-04-2024': 
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) 
            elif current_date == '13-04-2024': 
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + max_today
            elif current_date == '14-04-2024':
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72)
                if row['R3'] == '--':
                    df.at[idx, 'SCORE'] += (max_r3 - 72)
                else: 
                    df.at[idx, 'SCORE'] += (int(row['R3']) - 72)
                df.at[idx, 'SCORE'] += max_today
    
    
    df = df[['PLAYER', 'SCORE']].rename(columns = {'PLAYER': 'golfer_name', 'SCORE': 'score'})
    
    df["golfer_name"] = df["golfer_name"].replace({
            'Ludvig Åberg': 'Ludvig Aberg',
            'Byeong Hun An': 'Byeong-Hun An', 
            'Nicolai Højgaard': 'Nicolai Hojgaard',
            'Joaquín Niemann': 'Joaquin Niemann', 
            'Christo Lamprecht (a)': 'Christo Lamprecht', 
            'Jasper Stubbs (a)': 'Jasper Stubbs',
            'Neal Shipley (a)': 'Neal Shipley', 
            'Santiago de la Fuente (a)': 'Santiago De la Fuente', 
            'Stewart Hagestad (a)': 'Stewart Hagestad', 
            'Thorbjørn Olesen': 'Thorbjorn Olesen'
        })
        
    df["score"] = df["score"].replace({
        'E': '0', 
    })
    df.columns = df.columns.get_level_values(0)
    df = df.dropna()
    for idx, row in df.iterrows(): 
        df.at[idx, 'score'] = int(str(row['score']).strip('+'))

In [16]:
def get_masters_scores(): 
    url = 'https://www.espn.com/golf/leaderboard'
    headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
    response = requests.get(url, headers=headers)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find the table with the specified class
        table = soup.find("tbody", class_="Table__TBODY")
        
        # Check if the table was found
        
        # Extract the rows of the table
        rows = table.find_all("tr")
        
        # Initialize an empty list to store the table data
        data = []
        
        # Loop through each row and extract the data
        for row in rows:
            # Extract the cells (td) of the row
            cells = row.find_all("td")
            
            # Extract the text content of each cell and append to the data list
            row_data = [cell.get_text() for cell in cells]
            data.append(row_data)
        headers = soup.find_all('th')
        header_texts = [header.text.strip() for header in headers]
        # Convert the data list into a pandas DataFrame
        df = pd.DataFrame(data)
        available_columns = [col for col in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] if col in df.columns]
        df = df[available_columns]
        df.columns = [x for x in header_texts]
        
        # df['R3'] = df['R3'].replace('--', '0')
        # df['R4'] = df['R4'].replace('--', '0')

        # Filter DataFrame
        df = df.dropna(subset = ['PLAYER'])
        filtered_df = df[~df['PLAYER'].str.endswith('(a)')]
        filtered_df = filtered_df[filtered_df['SCORE'] != 'CUT']
        filtered_df["TODAY"] = filtered_df["TODAY"].replace({
                'E': '0', 
                '-': '0'
            })
        filtered_df["R3"] = filtered_df["R3"].replace({
                'E': '0', 
                '--': '0'
            })

        for idx, row in filtered_df.iterrows(): 
                filtered_df.at[idx, 'TODAY'] = int(str(row['TODAY']).strip('+'))
                filtered_df.at[idx, 'R3'] = int(str(row['R3']).strip('+'))
                
        max_today = filtered_df['TODAY'].max() 
        max_r3 = filtered_df['R3'].max()

        # If we are on Saturday (R3) only add R1 and R2 else add R1 R2 and R3 
        current_date = datetime.now().date().strftime('%d-%m-%Y')

        for idx, row in df.iterrows(): 
            if row['SCORE'] == 'CUT': 
                if current_date == '11-04-2025': 
                    df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) 
                elif current_date == '12-04-2025': 
                    df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + max_today
                elif current_date == '13-04-2025':
                    df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72)
                    if row['R3'] == '--':
                        df.at[idx, 'SCORE'] += (max_r3 - 72)
                    else: 
                        df.at[idx, 'SCORE'] += (int(row['R3']) - 72)
                    df.at[idx, 'SCORE'] += max_today
        
        
        df = df[['PLAYER', 'SCORE']].rename(columns = {'PLAYER': 'golfer_name', 'SCORE': 'score'})
        
        df["golfer_name"] = df["golfer_name"].replace({
                'Ángel Cabrera': 'Angel Cabrera',
                'Byeong Hun An': 'Byeong-Hun An',
                'Cam Davis': 'Cameron Davis',
                'Evan Beck (a)': 'Evan Beck',
                'Hiroshi Tai (a)': 'Hiroshi Tai',
                'Joaquín Niemann': 'Joaquin Niemann',
                'Jose Luis Ballester (a)': 'Jose Luis Ballester',
                'José María Olazábal': 'Jose Maria Olazabal',
                'Justin Hastings (a)': 'Justin Hastings',
                'Ludvig Åberg': 'Ludvig Aberg',
                'Matt McCarty': 'Matthew McCarty',
                'Nicolai Højgaard': 'Nicolai Hojgaard',
                'Noah Kent (a)': 'Noah Kent',
                'Rasmus Højgaard': 'Rasmus Hojgaard'
            })
            
        df["score"] = df["score"].replace({
            'E': '0', 
        })
        df.columns = df.columns.get_level_values(0)
        df = df.dropna()
        for idx, row in df.iterrows(): 
            df.at[idx, 'score'] = int(str(row['score']).strip('+'))
        return df 
    else:
        return f"Failed to retrieve ESPN scores. Status code:{response.status_code}"

def calculate_top_n(row, n):
    scores = []
    for col in ['tier_1_1_score', 'tier_1_2_score', 'tier_1_3_score',
                'tier_2_1_score', 'tier_2_2_score', 'tier_2_3_score',
                'tier_3_1_score', 'tier_3_2_score', 'tier_4_1_score']:
        try:
            # Replace NaN with 0 before converting to int
            score = 0 if pd.isna(row[col]) else int(row[col])
            scores.append(score)
        except (ValueError, KeyError):
            scores.append(0)  # Use 0 as default score for any errors
    
    return sum(sorted(scores)[:n])

scores = get_masters_scores()
print(scores)

picks = pd.read_csv('masters_picks.csv')
picks = picks.drop(columns = ['PAYMENT - Select Option Below & Pay Prior to Submission'])
picks = picks.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Merging golfers_df with masters_data_df on golfer names
for col in ['tier_1_1', 'tier_1_2', 'tier_1_3', 'tier_2_1', 'tier_2_2', 'tier_2_3', 'tier_3_1', 'tier_3_2', 'tier_4_1']: 
    if col == 'tier_1_1': 
        merged_df = pd.merge(picks, scores, how='left', left_on=col, right_on='golfer_name')
        merged_df = merged_df.drop(columns = 'golfer_name')
        merged_df = merged_df.rename(columns = {'score': f'{col}_score'})
    else: 
        merged_df = pd.merge(merged_df, scores, how ='left', left_on=col, right_on = 'golfer_name')
        merged_df = merged_df.drop(columns = ['golfer_name'])
        merged_df = merged_df.rename(columns = {'score': f'{col}_score'})

          golfer_name score
0         Justin Rose    -7
1       Corey Conners    -4
2   Scottie Scheffler    -4
3      Tyrrell Hatton    -3
4     Matthew McCarty    -3
..                ...   ...
90     Patton Kizzire     7
91          Noah Kent     7
92       Thomas Detry     7
93  Thriston Lawrence     7
94        Nick Dunlap    18

[95 rows x 2 columns]


In [17]:
merged_df

,name,tier_1_1,tier_1_2,tier_1_3,tier_2_1,tier_2_2,tier_2_3,tier_3_1,tier_3_2,tier_4_1,tier_1_1_score,tier_1_2_score,tier_1_3_score,tier_2_1_score,tier_2_2_score,tier_2_3_score,tier_3_1_score,tier_3_2_score,tier_4_1_score
0,Patrick Schramm,Rory McIlroy,Jon Rahm,Collin Morikawa,Tyrrell Hatton,Wyndham Clark,Sepp Straka,Billy Horschel,Joe Highsmith,Patton Kizzire,-3,3,0,-3,1,3,5,4,7
1,Chase Werry,Scottie Scheffler,Rory McIlroy,Bryson DeChambeau,Cameron Smith,Shane Lowry,Corey Conners,Daniel Berger,Billy Horschel,Matthew McCarty,-4,-3,-2,-1,-1,-4,-1,5,-3
2,Tyler Christian,Scottie Scheffler,Collin Morikawa,Tommy Fleetwood,Cameron Smith,Shane Lowry,Akshay Bhatia,Patrick Reed,Aaron Rai,Matthieu Pavon,-4,0,2,-1,-1,2,-1,-2,6
3,Tyler Christian,Scottie Scheffler,Collin Morikawa,Tommy Fleetwood,Tony Finau,Min Woo Lee,Akshay Bhatia,Sahith Theegala,Patrick Reed,Danny Willett,-4,0,2,3,-1,2,1,-1,3
4,James Callahan,Rory McIlroy,Ludvig Aberg,Hideki Matsuyama,Sungjae Im,Tony Finau,Russell Henley,Harris English,J.J. Spaun,Rafael Campos,-3,-1,-2,-1,3,3,-2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,James Young,Scottie Scheffler,Rory McIlroy,Brooks Koepka,Wyndham Clark,Sepp Straka,Cameron Young,Max Homa,Keegan Bradley,Bubba Watson,-4,-3,0,1,3,0,1,2,-1
513,Luke Krantz,Scottie Scheffler,Ludvig Aberg,Bryson DeChambeau,Min Woo Lee,Akshay Bhatia,Tom Kim,Max Homa,Billy Horschel,Kevin Yu,-4,-1,-2,-1,2,1,1,5,4
514,Ehren Webber,Scottie Scheffler,Rory McIlroy,Collin Morikawa,Shane Lowry,Russell Henley,Corey Conners,Keegan Bradley,Billy Horschel,Matthieu Pavon,-4,-3,0,-1,3,-4,2,5,6
515,Ehren Webber,Scottie Scheffler,Ludvig Aberg,Tommy Fleetwood,Tony Finau,Min Woo Lee,Russell Henley,Patrick Reed,Phil Mickelson,Jose Luis Ballester,-4,-1,2,3,-1,3,-1,3,4


In [13]:
picks

,name,tier_1_1,tier_1_2,tier_1_3,tier_2_1,tier_2_2,tier_2_3,tier_3_1,tier_3_2,tier_4_1
0,Patrick Schramm,Rory McIlroy,Jon Rahm,Collin Morikawa,Tyrrell Hatton,Wyndham Clark,Sepp Straka,Billy Horschel,Joe Highsmith,Patton Kizzire
1,Chase Werry,Scottie Scheffler,Rory McIlroy,Bryson DeChambeau,Cameron Smith,Shane Lowry,Corey Conners,Daniel Berger,Billy Horschel,Matthew McCarty
2,Tyler Christian,Scottie Scheffler,Collin Morikawa,Tommy Fleetwood,Cameron Smith,Shane Lowry,Akshay Bhatia,Patrick Reed,Aaron Rai,Matthieu Pavon
3,Tyler Christian,Scottie Scheffler,Collin Morikawa,Tommy Fleetwood,Tony Finau,Min Woo Lee,Akshay Bhatia,Sahith Theegala,Patrick Reed,Danny Willett
4,James Callahan,Rory McIlroy,Ludvig Aberg,Hideki Matsuyama,Sungjae Im,Tony Finau,Russell Henley,Harris English,J.J. Spaun,Rafael Campos
...,...,...,...,...,...,...,...,...,...,...
512,James Young,Scottie Scheffler,Rory McIlroy,Brooks Koepka,Wyndham Clark,Sepp Straka,Cameron Young,Max Homa,Keegan Bradley,Bubba Watson
513,Luke Krantz,Scottie Scheffler,Ludvig Aberg,Bryson DeChambeau,Min Woo Lee,Akshay Bhatia,Tom Kim,Max Homa,Billy Horschel,Kevin Yu
514,Ehren Webber,Scottie Scheffler,Rory McIlroy,Collin Morikawa,Shane Lowry,Russell Henley,Corey Conners,Keegan Bradley,Billy Horschel,Matthieu Pavon
515,Ehren Webber,Scottie Scheffler,Ludvig Aberg,Tommy Fleetwood,Tony Finau,Min Woo Lee,Russell Henley,Patrick Reed,Phil Mickelson,Jose Luis Ballester


In [66]:
# df['R3'] = df['R3'].replace('--', 0)
# df['R4'] = df['R4'].replace('--', 0)

# # Filter DataFrame
# df = df.dropna(subset = ['PLAYER'])
# filtered_df = df[~df['PLAYER'].str.endswith('(a)')]
# filtered_df = filtered_df[filtered_df['SCORE'] != 'CUT']
# filtered_df["TODAY"] = filtered_df["TODAY"].replace({
#         'E': '0', 
#     })

# for idx, row in filtered_df.iterrows(): 
#         filtered_df.at[idx, 'TODAY'] = int(str(row['TODAY']).strip('+'))
        
# max_today = filtered_df['TODAY'].max() 

# # If we are on Saturday (R3) only add R1 and R2 else add R1 R2 and R3 
# current_date = datetime.now().date().strftime('%d-%m-%Y')

# for idx, row in df.iterrows(): 
#     if row['SCORE'] == 'CUT': 
#         if current_date == '12-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) 
#         elif current_date == '13-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + max_today
#         elif current_date == '14-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + (int(row['R3']) - 72) + max_today
    

In [3]:
df

,golfer_name,score
0,Justin Rose,-6
1,Corey Conners,-4
2,Scottie Scheffler,-4
3,Tyrrell Hatton,-3
4,Aaron Rai,-2
...,...,...
90,Patton Kizzire,7
91,Noah Kent (a),7
92,Thriston Lawrence,7
93,Matthieu Pavon,7


In [4]:
filtered_df['TODAY']

0     -6
1     -4
2     -4
3     -3
4     -2
      ..
89     6
90     7
92     7
93     7
94    18
Name: TODAY, Length: 90, dtype: object